In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping


In [16]:
# Datos de ejemplo (secuencia de números)
df = pd.read_excel('/home/waldos/Documents/2doCodigo/RecPatron/BD/DatosFrame/Datos frames.xlsx')  # Reemplaza 'datos.xlsx' con el nombre de tu archivo Excel

In [17]:
df.head()

,Nombre imagen,Biomasa g/L
0,roi_000,0.150
1,roi_001,0.165
2,roi_002,0.177
3,roi_003,0.145
4,roi_004,0.158


In [18]:
data = df['Biomasa g/L'].values

In [19]:
print(data)

[0.15  0.165 0.177 ... 3.333 3.    3.333]


In [20]:
# Preparar datos para LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float32), tf.convert_to_tensor(y, dtype=tf.float32)


In [21]:
# Definir hiperparámetros
n_steps = 4
n_features = 1

# Preparar datos
X, y = prepare_data(data, n_steps)

In [22]:
print(X)


tf.Tensor(
[[0.15  0.165 0.177 0.145]
 [0.165 0.177 0.145 0.158]
 [0.177 0.145 0.158 0.125]
 ...
 [3.    3.333 3.333 3.333]
 [3.333 3.333 3.333 3.333]
 [3.333 3.333 3.333 3.   ]], shape=(1076, 4), dtype=float32)


In [23]:
print(y)

tf.Tensor([0.158 0.125 0.129 ... 3.333 3.    3.333], shape=(1076,), dtype=float32)


In [24]:
# Definir el modelo
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [25]:
# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X, y, epochs=100, verbose=1, batch_size=1, validation_split=0.2, callbacks=[early_stopping])




Epoch 1/100
860/860 [==============================] - 7s 6ms/step - loss: 0.1068 - val_loss: 0.0311
Epoch 2/100
860/860 [==============================] - 5s 6ms/step - loss: 0.0188 - val_loss: 0.0304
Epoch 3/100
860/860 [==============================] - 5s 6ms/step - loss: 0.0197 - val_loss: 0.0177
Epoch 4/100
860/860 [==============================] - 5s 6ms/step - loss: 0.0175 - val_loss: 0.0204
Epoch 5/100
860/860 [==============================] - 5s 6ms/step - loss: 0.0165 - val_loss: 0.0172
Epoch 6/100
860/860 [==============================] - 5s 6ms/step - loss: 0.0167 - val_loss: 0.0176
Epoch 7/100
860/860 [==============================] - 5s 6ms/step - loss: 0.0165 - val_loss: 0.0291
Epoch 8/100
860/860 [==============================] - 5s 6ms/step - loss: 0.0164 - val_loss: 0.0181


In [27]:
# Hacer predicciones
x_input = tf.convert_to_tensor([[i] for i in range(100, 104)])
x_input = tf.reshape(x_input, (1, n_steps, n_features))
y_pred = model.predict(x_input, verbose=0)

print("Predicción:", y_pred[0][0])

Predicción: 94.17955
